In [2]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

Populating the interactive namespace from numpy and matplotlib


# Exploring the relationship between low/high tapasin expressers and HLA genotype <br>
Hypothesis: Since the functional relationship between these genes is important and they come from the same region of the genome, we think there may be a germline relationship <br>
Approach: Test all HLA genotypes with expression to see if there is enrichement (fisher's exact test) <br>
Data: MHC-I/MHC-II typing; tapasin expression is from Su Xian

Tapasin expression groups

In [16]:
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/clinical/TAPBP/TAPBP_expression.csv', header=None, sep='\t')
df.columns = ['Patient', 'TAPBP']

In [17]:
low_TAPBP_expr_digits = [x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/clinical/TAPBP/Low_TAPBP.txt').readlines()]
high_TAPBP_expr_digits = [x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/clinical/TAPBP/High_TAPBP.txt').readlines()]

In [18]:
print len(low_TAPBP_expr_digits), len(high_TAPBP_expr_digits)

1533 6940


HLA genotypes

In [19]:
classI_types = pd.read_csv('/cellar/users/ramarty/Data/hla/hla_typing/sample_and_types.with_homomzygous.csv', index_col=0)
classI_types.index = list(classI_types['Sample'])
classII_types = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/hla_types/hla_types.tcga.csv', index_col=0)

Restricting high/low tapasin expression to patients with types

In [20]:
restricted = classI_types[['Sample']]
def get_last_four_digits(x):
    return x[-4:]
restricted['last_four'] = classI_types['Sample'].apply(get_last_four_digits)
restricted = restricted[restricted.last_four.isin(low_TAPBP_expr_digits+high_TAPBP_expr_digits)]
def get_expression(x):
    if x in low_TAPBP_expr_digits:
        return 'low'
    elif x in high_TAPBP_expr_digits:
        return 'high'
    else:
        return 'med'
restricted['expression'] = restricted.last_four.apply(get_expression)

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
restricted.expression.value_counts()

high    6140
low     1374
Name: expression, dtype: int64

In [22]:
low_TAPBP_expr_patients = list(restricted[restricted.expression == 'low'].Sample)
high_TAPBP_expr_patients = list(restricted[restricted.expression == 'high'].Sample)

Actual analysis

In [23]:
all_alleles = list(set(list(classI_types.A1)+list(classI_types.A2)+
                      list(classI_types.B1)+list(classI_types.B2)+
                      list(classI_types.C1)+list(classI_types.C2)))
print len(all_alleles)

245


In [24]:
len(low_TAPBP_expr_patients), len(high_TAPBP_expr_patients)

(1374, 6140)

In [25]:
len(restricted)

7514

In [28]:
low_TAPBP_expr_patients = list(restricted.sample(1374).Sample)
high_TAPBP_expr_patients = [x for x in list(restricted.Sample) if x not in low_TAPBP_expr_patients]
print len(low_TAPBP_expr_patients), len(high_TAPBP_expr_patients)

1374 6140


In [29]:
ORs, Ps, alleles_used, patient_frequency = [], [], [], []
for i, allele in enumerate(all_alleles):
    
    if i % 10 == 0:
        print i
    def allele_present(x):
        if allele in list(x):
            return True
        else:
            return False
    classI_types['allele_present'] = classI_types[['A1', 'A2', 'B1', 'B2', 'C1', 'C2']].apply(allele_present, axis=1)
    
    patients_with_allele = list(classI_types[classI_types.allele_present].index)
    
    if len(patients_with_allele) > 25:
        patients_without_allele = list(classI_types[~classI_types.allele_present].index)
    
        OR, P = sp.fisher_exact([[len([x for x in low_TAPBP_expr_patients if x in patients_with_allele]),
                          len([x for x in low_TAPBP_expr_patients if x in patients_without_allele])],
                         [len([x for x in high_TAPBP_expr_patients if x in patients_with_allele]),
                          len([x for x in high_TAPBP_expr_patients if x in patients_without_allele])]])
        ORs.append(OR)
        Ps.append(P)
        alleles_used.append(allele)
        patient_frequency.append(float(len(patients_with_allele))/(len(patients_with_allele)+len(patients_without_allele)))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240


In [30]:
results_df = pd.DataFrame({'Allele': alleles_used,
                           'OR': ORs, 
                           'P': Ps,
                           'Allele_frequency': patient_frequency})
print len(results_df)

112


In [31]:
stats = importr('stats')
results_df['P_BH_adjusted'] = stats.p_adjust(FloatVector(list(results_df.P)), method = 'BH')

In [33]:
results_df.sort_values('P_BH_adjusted')

,Allele,Allele_frequency,OR,P,P_BH_adjusted
111,C*02:02,0.079988,1.266524,0.027259,0.559564
68,A*02:07,0.007115,1.891946,0.049324,0.559564
71,A*02:02,0.009351,0.423072,0.040732,0.559564
93,B*56:01,0.013619,1.684477,0.019531,0.559564
30,C*08:02,0.060169,0.758736,0.041565,0.559564
49,A*30:02,0.025612,0.627990,0.032850,0.559564
48,B*46:01,0.009757,1.833936,0.023910,0.559564
23,B*18:01,0.092591,1.215660,0.049961,0.559564
83,B*44:05,0.005082,2.336370,0.018399,0.559564
14,B*39:06,0.016465,1.575780,0.044486,0.559564


In [34]:
results_df.sort_values('P_BH_adjusted').to_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/TAPBP/tapasin_hla_genotype.fisher_exact_results.randomized.csv')